In [2]:
print("Ok")

Ok


In [3]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_pinecone import PineconeVectorStore
from langchain.vectorstores.pinecone import PineconeRetriever
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from pinecone import Pinecone
from dotenv import load_dotenv
import pypdf
import os




c:\Users\user\anaconda3\envs\ansibot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
load_dotenv()

True

In [5]:
PINECONE_API_KEY = "92c9671d-43cc-48e8-9a52-c60bc89ff702"

In [6]:
print(PINECONE_API_KEY)

92c9671d-43cc-48e8-9a52-c60bc89ff702


In [7]:
data=r"C:\Users\user\Documents\Github\projects\LlamaAi\AnsiAssistance\data"

In [8]:
def load_pdf(data):
    loader = DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents


In [9]:
#Extract data from the pdf
documents = load_pdf(data)

In [10]:
documents

[Document(metadata={'source': 'C:\\Users\\user\\Documents\\Github\\projects\\LlamaAi\\AnsiAssistance\\data\\E2624-17-SP-Torque-Calibration-Testing-Machine.pdf', 'page': 0}, page_content="Designation: E2624 −17\nStandard Practice for\nTorque Calibration of Testing Machines1\nThis standard is issued under the ﬁxed designation E2624; the number immediately following the designation indicates the year of\noriginal adoption or, in the case of revision, the year of last revision. A number in parentheses indicates the year of last reapproval. Asuperscript epsilon ( ´) indicates an editorial change since the last revision or reapproval.\n1. Scope*\n1.1 This practice covers procedures and requirements for\nthe calibration of torque for static and quasi-static torquecapable testing machines. These may, or may not, have torqueindicating systems and include those devices used for thecalibration of hand torque tools. Testing machines may becalibrated by one of the three following methods or combina

In [11]:
def text_split(extracted_data):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
  text_chunks = text_splitter.split_documents(extracted_data)
  return text_chunks

In [12]:
text_chunks = text_split(documents)

In [13]:
text_chunks

[Document(page_content='Designation: E2624 −17\nStandard Practice for\nTorque Calibration of Testing Machines1\nThis standard is issued under the ﬁxed designation E2624; the number immediately following the designation indicates the year of\noriginal adoption or, in the case of revision, the year of last revision. A number in parentheses indicates the year of last reapproval. Asuperscript epsilon ( ´) indicates an editorial change since the last revision or reapproval.\n1. Scope*', metadata={'source': 'C:\\Users\\user\\Documents\\Github\\projects\\LlamaAi\\AnsiAssistance\\data\\E2624-17-SP-Torque-Calibration-Testing-Machine.pdf', 'page': 0}),
 Document(page_content='1. Scope*\n1.1 This practice covers procedures and requirements for\nthe calibration of torque for static and quasi-static torquecapable testing machines. These may, or may not, have torqueindicating systems and include those devices used for thecalibration of hand torque tools. Testing machines may becalibrated by one of t

In [14]:
print(len(text_chunks))

122


In [15]:
#download embedding model
def download_hugging_face_embedding():
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
  return embeddings

In [16]:
embeddings = download_hugging_face_embedding()

c:\Users\user\anaconda3\envs\ansibot\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [17]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [18]:
query_result = embeddings.embed_query("Hello World")

In [19]:
query_result

[-0.03447726368904114,
 0.031023195013403893,
 0.006734934169799089,
 0.026109030470252037,
 -0.039362065494060516,
 -0.16030246019363403,
 0.06692402064800262,
 -0.006441517733037472,
 -0.04745056480169296,
 0.014758898876607418,
 0.07087533175945282,
 0.05552754923701286,
 0.019193338230252266,
 -0.02625126764178276,
 -0.01010944414883852,
 -0.02694048173725605,
 0.022307468578219414,
 -0.022226648405194283,
 -0.1496926099061966,
 -0.017493069171905518,
 0.007676312699913979,
 0.0543522983789444,
 0.003254531417042017,
 0.03172597661614418,
 -0.084621362388134,
 -0.029405953362584114,
 0.05159563198685646,
 0.04812399297952652,
 -0.0033147777430713177,
 -0.058279190212488174,
 0.041969332844018936,
 0.02221066504716873,
 0.1281888484954834,
 -0.02233894355595112,
 -0.011656250804662704,
 0.06292835623025894,
 -0.032876234501600266,
 -0.09122610837221146,
 -0.03117542713880539,
 0.05269954353570938,
 0.047034844756126404,
 -0.0842030718922615,
 -0.030056234449148178,
 -0.0207447521388

In [20]:
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

index_name = "ansichatbot"

index = pc.Index(index_name)



In [21]:
vectorstore = PineconeVectorStore(index=index,embedding=embeddings)


In [22]:
vectorstore.add_texts([t.page_content for t in text_chunks])

['a2a618c1-f5b3-453b-9761-5be3c6410d65',
 '4e024278-1c6a-42e6-8779-0c69bf4195b3',
 'f570fab1-3e5c-48f3-a634-2416c3b6bcd9',
 '38e221e0-5e49-4f34-9964-eb91af37556f',
 '86573669-3dfe-459b-99f8-e731f7ce573f',
 '6a01f4fc-cc39-4d13-baac-677434b4adc6',
 'e3e6ecf2-1532-4fd6-a82b-258f67d63b22',
 'a3fd63cf-12ab-4097-bbee-94431cc2ebf5',
 '593b16cf-b035-4ea1-b92a-976e162d65cc',
 '9473972c-b678-4ef5-b18e-82912eb106b8',
 'ad8cbb1f-11d9-44f8-90f6-0fed101dc392',
 'a71fcac9-b07c-4d1a-9bb1-408bdf60882c',
 '685ad70b-8c9c-4d65-b944-9f4bfa4e94c0',
 '092a4ab0-1c24-4c11-9520-19c2d3207eba',
 '642ffaab-f30f-4367-970f-5bc4fb4d4478',
 '83ad0957-0e25-4811-bfaf-5d1d54c0016c',
 'f9d9328e-a0b6-4ca9-af4f-e0ff9d27fc78',
 '1dc1fbf8-900c-4470-8b35-88717333b5c6',
 '05f3d638-64f8-420a-81dc-736fac26e09a',
 '8330c6f3-aba0-4757-821a-6130506163b8',
 '403bd704-5f9c-44f1-a469-ff8f2cff4a7f',
 '60f461b0-9fd0-48fd-bb4a-490b63687db9',
 '1a1c4ed4-e945-46e2-bceb-a8fc38a6d094',
 '5444a5fb-53c8-4399-8d45-feef830f658e',
 'a2845b72-2928-

In [32]:

retriever = vectorstore.as_retriever()
print(type(retriever))

<class 'langchain_core.vectorstores.base.VectorStoreRetriever'>


In [24]:
query = "How to calibrate torque?"

results = vectorstore.similarity_search(query=query,k=5)
for doc in results:
    print(doc)


page_content='Calibration by the application of standard weights using a leverarm to the torque sensing mechanism of the testing machine,where practicable, is the most accurate method. Its limitationsare: (1)the small range of torque that can be calibrated, (2)the
non-portability of any high capacity standard weights and (3)
analysis of all parasitic torque components.
5.2 Calibration by Elastic Calibration Devices— The sec-'
page_content='Calibration by the application of standard weights using a leverarm to the torque sensing mechanism of the testing machine,where practicable, is the most accurate method. Its limitationsare: (1)the small range of torque that can be calibrated, (2)the
non-portability of any high capacity standard weights and (3)
analysis of all parasitic torque components.
5.2 Calibration by Elastic Calibration Devices— The sec-'
page_content='Calibration by the application of standard weights using a leverarm to the torque sensing mechanism of the testing machine,whe

In [25]:
prompt_template = """

Use the following pieces of information to answer the user´s question.
if you don´t know the answer, just say that you don´t know, don´t try to make up an answer.

Context:{context}
Question:{question}

only return the helpful answer below and nothing else.
Helpful answer:
"""

In [26]:
PROMPT = PromptTemplate(
  template=prompt_template,
  input_variables=["context","question"],
)
chain_type_kwargs={"prompt":PROMPT}

In [27]:
llm = CTransformers(
  model=r"C:\Users\user\Documents\Github\projects\LlamaAi\AnsiAssistance\Model\Llama 2\llama-2-7b-chat.ggmlv3.q4_0.bin",
  model_type="llama",
  config={
    "max_new_tokens":512,
    "temperature":0.8
  }
  
)

In [28]:
query = "What is torque?"


In [31]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)

ValidationError: 1 validation error for RetrievalQA
retriever
  Can't instantiate abstract class BaseRetriever with abstract methods _aget_relevant_documents, _get_relevant_documents (type=type_error)

In [267]:
while True:
    user_input = input(f"Input Prompt:")
    results = qa({"query":user_input})
    print ("Response : ", results["result"])

NameError: name 'qa' is not defined